In [204]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory').text

In [4]:
soup = BeautifulSoup(website_url,'lxml')

In [5]:
table = soup.find('table',{'class':'wikitable'})

In [14]:
url='https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory'

df=pd.read_html(url, header=0)[1]
df.head()

,Country,Official language,Regional language,Minority language,National language,Widely spoken
0,Abkhazia[a],Abkhaz Russian,NaN,Georgian,Abkhaz,NaN
1,Afghanistan[1],Pashto Dari,Uzbek[b] Turkmen[b] Pashai[b] Nuristani[b] Bal...,NaN,NaN,NaN
2,Albania[2],Albanian,NaN,Greek Macedonian Aromanian,NaN,Italian
3,Algeria[3],Arabic Tamazight,NaN,NaN,Arabic Tamazight,French
4,Andorra,Catalan[4],NaN,Spanish French Portuguese,NaN,NaN


In [15]:
df = df[['Country','Official language','National language','Widely spoken']]

In [16]:
import re

#regex = re.compile('[^a-zA-Z]')

df['country'] = df['Country'].apply(lambda x: re.sub('[\(\[].*?[\)\]]', '', x))
df[['Official language','National language','Widely spoken']] = df[['Official language','National language','Widely spoken']].astype(str)
df['Official language'] = df['Official language'].apply(lambda x: '' if x == "nan" else x)
df['National language'] = df['National language'].apply(lambda x: '' if x == "nan" else x)
df['Widely spoken'] = df['Widely spoken'].apply(lambda x: '' if x == "nan" else x)
df['language'] = df[['Official language','National language','Widely spoken']].apply(lambda x: ' '.join(x), axis=1)
df['language'] = df['language'].apply(lambda x: re.sub('[\(\[].*?[\)\]]', '', x))

df = df[['country','language']]

In [17]:
df.head()

,country,language
0,Abkhazia,Abkhaz Russian Abkhaz
1,Afghanistan,Pashto Dari
2,Albania,Albanian Italian
3,Algeria,Arabic Tamazight Arabic Tamazight French
4,Andorra,Catalan


In [18]:
url2='https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)'

df2=pd.read_html(url2, header=0)[3]

In [19]:
df2.head()

,Country or area,UN continentalregion[4],UN statisticalregion[4],Population(1 July 2018),Population(1 July 2019),Change
0,China[a],Asia,Eastern Asia,1427647786,1433783686,+0.43%
1,India,Asia,Southern Asia,1352642280,1366417754,+1.02%
2,United States,Americas,Northern America,327096265,329064917,+0.60%
3,Indonesia,Asia,South-eastern Asia,267670543,270625568,+1.10%
4,Pakistan,Asia,Southern Asia,212228286,216565318,+2.04%


In [20]:
df2['country'] = df2['Country or area'].apply(lambda x: re.sub('[\(\[].*?[\)\]]', '', x))

In [21]:
df2['pop'] = df2['Population(1 July 2019)']
df2 = df2[['country','pop']]
df2.head(10)

,country,pop
0,China,1433783686
1,India,1366417754
2,United States,329064917
3,Indonesia,270625568
4,Pakistan,216565318
5,Brazil,211049527
6,Nigeria,200963599
7,Bangladesh,163046161
8,Russia,145872256
9,Mexico,127575529


In [22]:
url3='https://en.wikipedia.org/wiki/List_of_countries_by_population_growth_rate'

df3=pd.read_html(url3, header=0)[2]

In [23]:
df3.head()

,Country/ Region,WB[1]2009(%),CIA WF[2]2012(%),CIA WF[3]2014(%),UN[4]2005-10(%),UN[4]2010-15(%),UN[4]2015-20(%)
0,Afghanistan,2.44,2.29,2.29,2.78,3.16,2.41
1,Albania,0.26,0.30,0.30,-0.92,-0.12,0.13
2,Algeria,1.89,1.88,1.88,1.63,1.98,1.67
3,Andorra,0.63,0.17,0.17,1.37,-1.59,-0.21
4,Angola,3.12,2.78,2.78,3.57,3.52,3.28


In [24]:
df3['country'] = df3['Country/ Region'].apply(lambda x: re.sub('[\(\[].*?[\)\]]', '', x))
df3.columns = ['country/region','2009','2012','2014','2005_10','2010_15','2015_20','country']
df3 = df3[['country','2005_10','2010_15','2015_20']]
df3.head()

,country,2005_10,2010_15,2015_20
0,Afghanistan,2.78,3.16,2.41
1,Albania,-0.92,-0.12,0.13
2,Algeria,1.63,1.98,1.67
3,Andorra,1.37,-1.59,-0.21
4,Angola,3.57,3.52,3.28


In [25]:
df4 = df.set_index('country').join(df2.set_index('country'))
df4 = df4.join(df3.set_index('country'))
df4 = df4[~df4['pop'].isnull()]
df4 = df4.reset_index()

In [32]:
df4[df4.language == '']

,country,language,pop,2005_10,2010_15,2015_20


In [33]:
df4

,country,language,pop,2005_10,2010_15,2015_20
0,Afghanistan,Pashto Dari,38041754.0,2.78,3.16,2.41
1,Albania,Albanian Italian,2880917.0,-0.92,-0.12,0.13
2,Algeria,Arabic Tamazight Arabic Tamazight French,43053054.0,1.63,1.98,1.67
3,Andorra,Catalan,77142.0,1.37,-1.59,-0.21
4,Angola,Portuguese Kimbundu Umbundu Kikongo Chokwe Kwa...,31825295.0,3.57,3.52,3.28
...,...,...,...,...,...,...
189,Venezuela,Spanish Venezuelan Sign Language,28515829.0,1.61,1.41,1.26
190,Vietnam,Vietnamese,96462106.0,0.96,1.12,1.00
191,Yemen,Arabic,29161922.0,2.74,2.62,2.33
192,Zambia,English,17861030.0,2.78,3.01,2.97


In [93]:
languages = pd.read_csv('languages.csv',header=None)
languages = languages[0]
languages = languages.apply(lambda x: re.sub('[\(\[].*?[\)\]]', '', x))
languages = languages.tolist()

languages

['Mandarin Chinese',
 'English',
 'Hindustani ',
 'Spanish',
 'Arabic',
 'Malay ',
 'Russian',
 'Bengali',
 'Portuguese',
 'French',
 'Hausa',
 'Punjabi',
 'Japanese',
 'German',
 'Persian',
 'Swahili',
 'Telugu',
 'Javanese',
 'Wu Chinese ',
 'Korean',
 'Tamil',
 'Cantonese',
 'Turkish',
 'Vietnamese',
 'Italian']

In [94]:
languages[0] = 'Mandarin'
languages[2] = 'Hindi'

In [95]:
df4[df4.language.str.contains('Telugu')]

,country,language,pop,2005_10,2010_15,2015_20


In [96]:
df4.count() #need to deal with nan growth rates

country     194
language    194
pop         194
2005_10     191
2010_15     191
2015_20     190
dtype: int64

In [97]:
for lang in languages:
    print(lang + ": " + str(len(df4[df4.language.str.contains(lang)])))
# need to deal with languages with 0 countries speaking them
# potential solution: combine official languages with national and widely spoken languages
# also need to deal with dialects of Chinese -- combine Mandarin with Wu Chinese?
# Telugu ?

Mandarin: 3
English: 64
Hindi: 2
Spanish: 20
Arabic: 29
Malay : 3
Russian: 8
Bengali: 1
Portuguese: 10
French: 30
Hausa: 2
Punjabi: 1
Japanese: 1
German: 7
Persian: 2
Swahili: 5
Telugu: 0
Javanese: 1
Wu Chinese : 0
Korean: 2
Tamil: 2
Cantonese: 2
Turkish: 2
Vietnamese: 2
Italian: 7


In [205]:
# For each of the language, get countries that speak that langauge (df4.language.str.contains(lang), sort, 
# and pick the first row, then append to a larger df

data = pd.DataFrame()
unique_countries = []

for lang in languages:
    temp = df4[df4.language.str.contains(lang)]
    #print(temp)
    if len(temp) != 0:
        temp = temp.sort_values(by='pop',ascending=False)
        for c in temp['country']:
            if c in unique_countries:
                print(c)
                temp = temp[temp['country'] != c]
        temp['lang'] = lang
        data = data.append(temp.iloc[:2])
        unique_countries = data['country'].tolist()

India
Philippines
Iran
Singapore


In [206]:
data = data.reset_index()
data = data[['country','lang','language','pop','2005_10','2010_15','2015_20']]
data[['2005_10','2010_15','2015_20']] = data[['2005_10','2010_15','2015_20']] / 100
data['capacity'] = data['pop']*1.5
data.loc[data['country'] == 'Hong Kong','2015_20'] = (0.83 / 100)
data.loc[data['country'] == 'Macau','2015_20'] = (1.69 / 100)

data

,country,lang,language,pop,2005_10,2010_15,2015_20,capacity
0,China,Mandarin,Mandarin,1433783686.0000,0.0057,0.0054,0.0039,2150675529.0000
1,Taiwan,Mandarin,Mandarin Formosan languages Hakka Mandarin Ta...,23773876.0000,0.0044,0.0033,0.0028,35660814.0000
2,India,English,Hindi English,1366417754.0000,0.0146,0.0123,0.0110,2049626631.0000
3,United States,English,English,329064917.0000,0.0090,0.0072,0.0071,493597375.5000
4,Fiji,Hindi,English Fijian Fiji Hindi,889953.0000,0.0091,0.0074,0.0072,1334929.5000
5,Mexico,Spanish,Spanish,127575529.0000,0.0157,0.0141,0.0123,191363293.5000
6,Philippines,Spanish,English Filipino Filipino Sign Language Filipi...,108116615.0000,0.0166,0.0164,0.0151,162174922.5000
7,Egypt,Arabic,Arabic,100388073.0000,0.0182,0.0218,0.0187,150582109.5000
8,Iran,Arabic,Persian Persian Persian Arabic,82913906.0000,0.0114,0.0125,0.0104,124370859.0000
9,Malaysia,Malay,Malay,31949777.0000,0.0183,0.0178,0.0135,47924665.5000


In [207]:
# For each of the unique country, need to:
# run a regression model to get r(t), 
# double the population to get carrying capacity
# then run a logistic growth model to get pop in 50yrs
# then group by language (sum the pop if countries are different) 

import sklearn
from sklearn.linear_model import LinearRegression

coeffs = []
intercepts = []

X = np.array([0,5,10]).reshape(-1,1)
for _, row in data.iterrows():
    y = np.array(row[['2005_10','2010_15','2015_20']])
    reg = LinearRegression().fit(X, y)
    coeffs.append(reg.coef_[0])
    intercepts.append(reg.intercept_)

data['r_coef'] = coeffs
data['r_intercept'] = intercepts

data

,country,lang,language,pop,2005_10,2010_15,2015_20,capacity,r_coef,r_intercept
0,China,Mandarin,Mandarin,1433783686.0000,0.0057,0.0054,0.0039,2150675529.0000,-0.0002,0.0059
1,Taiwan,Mandarin,Mandarin Formosan languages Hakka Mandarin Ta...,23773876.0000,0.0044,0.0033,0.0028,35660814.0000,-0.0002,0.0043
2,India,English,Hindi English,1366417754.0000,0.0146,0.0123,0.0110,2049626631.0000,-0.0004,0.0144
3,United States,English,English,329064917.0000,0.0090,0.0072,0.0071,493597375.5000,-0.0002,0.0087
4,Fiji,Hindi,English Fijian Fiji Hindi,889953.0000,0.0091,0.0074,0.0072,1334929.5000,-0.0002,0.0089
5,Mexico,Spanish,Spanish,127575529.0000,0.0157,0.0141,0.0123,191363293.5000,-0.0003,0.0157
6,Philippines,Spanish,English Filipino Filipino Sign Language Filipi...,108116615.0000,0.0166,0.0164,0.0151,162174922.5000,-0.0001,0.0168
7,Egypt,Arabic,Arabic,100388073.0000,0.0182,0.0218,0.0187,150582109.5000,0.0001,0.0193
8,Iran,Arabic,Persian Persian Persian Arabic,82913906.0000,0.0114,0.0125,0.0104,124370859.0000,-0.0001,0.0119
9,Malaysia,Malay,Malay,31949777.0000,0.0183,0.0178,0.0135,47924665.5000,-0.0005,0.0189


In [214]:
# discrete logistic growth:
# n[t+1] = (1+r[t](1 - n[t]/K))*n[t]

#data[['pop','capacity']] = data #represent pop in million units
pop_in_50_yrs = []



def logistic_growth(n,r,k):
    return (1 + r*(1-n/k))*n

for _, row in data[['pop','capacity','r_coef','r_intercept']].iterrows():
    row[['pop','capacity']] =  row[['pop','capacity']] / (10**3)
    n = row['pop']
    k = row['capacity']
    r_coef = row['r_coef']
    r_intercept = row['r_intercept']
    
    for t in range(11,61,5):
        r_t = r_coef*t + r_intercept
        n = logistic_growth(n,r_t,k)
        if n<0: n=0

    pop_in_50_yrs.append(n)
    
pop_50 = [i * (10**3) for i in pop_in_50_yrs]

In [215]:
data['pop_in_50'] = pop_50
data

,country,lang,language,pop,2005_10,2010_15,2015_20,capacity,r_coef,r_intercept,pop_in_50,avg_r,pop_in_50_const_r
0,China,Mandarin,Mandarin,1433783686.0000,0.0057,0.0054,0.0039,2150675529.0000,-0.0002,0.0059,1433167611.3796,0.0050,1457497919.1478
1,Taiwan,Mandarin,Mandarin Formosan languages Hakka Mandarin Ta...,23773876.0000,0.0044,0.0033,0.0028,35660814.0000,-0.0002,0.0043,23689811.6173,0.0035,24049764.9407
2,India,English,Hindi English,1366417754.0000,0.0146,0.0123,0.0110,2049626631.0000,-0.0004,0.0144,1377209458.2658,0.0126,1422824690.5739
3,United States,English,English,329064917.0000,0.0090,0.0072,0.0071,493597375.5000,-0.0002,0.0087,331636620.0485,0.0078,337482289.6956
4,Fiji,Hindi,English Fijian Fiji Hindi,889953.0000,0.0091,0.0074,0.0072,1334929.5000,-0.0002,0.0089,897300.7759,0.0079,913103.5939
5,Mexico,Spanish,Spanish,127575529.0000,0.0157,0.0141,0.0123,191363293.5000,-0.0003,0.0157,129412041.6788,0.0140,133412002.0263
6,Philippines,Spanish,English Filipino Filipino Sign Language Filipi...,108116615.0000,0.0166,0.0164,0.0151,162174922.5000,-0.0001,0.0168,112276852.9624,0.0160,113748843.9460
7,Egypt,Arabic,Arabic,100388073.0000,0.0182,0.0218,0.0187,150582109.5000,0.0001,0.0193,107176953.3715,0.0196,106731707.9924
8,Iran,Arabic,Persian Persian Persian Arabic,82913906.0000,0.0114,0.0125,0.0104,124370859.0000,-0.0001,0.0119,85254852.4532,0.0114,86017665.7499
9,Malaysia,Malay,Malay,31949777.0000,0.0183,0.0178,0.0135,47924665.5000,-0.0005,0.0189,32253202.9374,0.0165,33664620.9784


In [ ]:
## maybe try cublic spine** either that or get more data

In [212]:
data['avg_r'] = np.mean(data[['2005_10','2010_15','2015_20']],axis=1)
data

,country,lang,language,pop,2005_10,2010_15,2015_20,capacity,r_coef,r_intercept,pop_in_50,avg_r
0,China,Mandarin,Mandarin,1433783686.0000,0.0057,0.0054,0.0039,2150675529.0000,-0.0002,0.0059,1422055393.1034,0.0050
1,Taiwan,Mandarin,Mandarin Formosan languages Hakka Mandarin Ta...,23773876.0000,0.0044,0.0033,0.0028,35660814.0000,-0.0002,0.0043,23221407.1680,0.0035
2,India,English,Hindi English,1366417754.0000,0.0146,0.0123,0.0110,2049626631.0000,-0.0004,0.0144,1403658013.4867,0.0126
3,United States,English,English,329064917.0000,0.0090,0.0072,0.0071,493597375.5000,-0.0002,0.0087,339705838.7382,0.0078
4,Fiji,Hindi,English Fijian Fiji Hindi,889953.0000,0.0091,0.0074,0.0072,1334929.5000,-0.0002,0.0089,920639.8945,0.0079
5,Mexico,Spanish,Spanish,127575529.0000,0.0157,0.0141,0.0123,191363293.5000,-0.0003,0.0157,135127468.9735,0.0140
6,Philippines,Spanish,English Filipino Filipino Sign Language Filipi...,108116615.0000,0.0166,0.0164,0.0151,162174922.5000,-0.0001,0.0168,126549185.5079,0.0160
7,Egypt,Arabic,Arabic,100388073.0000,0.0182,0.0218,0.0187,150582109.5000,0.0001,0.0193,128360513.3869,0.0196
8,Iran,Arabic,Persian Persian Persian Arabic,82913906.0000,0.0114,0.0125,0.0104,124370859.0000,-0.0001,0.0119,93613910.2051,0.0114
9,Malaysia,Malay,Malay,31949777.0000,0.0183,0.0178,0.0135,47924665.5000,-0.0005,0.0189,32946563.3861,0.0165


In [217]:
pop_in_50_yrs = []

for _, row in data[['pop','capacity','avg_r']].iterrows():
    #row[['pop','capacity']] =  row[['pop','capacity']] / (10**3)
    n = (row['pop'] / float(10**3))
    k = (row['capacity'] / float(10**3))
    #r_coef = row['r_coef']
    #r_intercept = row['r_intercept']
    r = row['avg_r']
    
    for t in range(11,61,5):
        #r_t = r_coef*t + r_intercept
        n = logistic_growth(n,r,k)
        if n<0: n=0

    pop_in_50_yrs.append(n)
    
pop_50 = [i * float(10**3) for i in pop_in_50_yrs]
data['pop_in_50_const_r'] = pop_50

data[['pop','capacity','pop','pop_in_50_const_r']] = data[['pop','capacity','pop','pop_in_50_const_r']].astype(int)
data

,country,lang,language,pop,2005_10,2010_15,2015_20,capacity,r_coef,r_intercept,pop_in_50,avg_r,pop_in_50_const_r
0,China,Mandarin,Mandarin,1433783686,0.0057,0.0054,0.0039,2150675529,-0.0002,0.0059,1433167611.3796,0.0050,1457497919
1,Taiwan,Mandarin,Mandarin Formosan languages Hakka Mandarin Ta...,23773876,0.0044,0.0033,0.0028,35660814,-0.0002,0.0043,23689811.6173,0.0035,24049764
2,India,English,Hindi English,1366417754,0.0146,0.0123,0.0110,2049626631,-0.0004,0.0144,1377209458.2658,0.0126,1422824690
3,United States,English,English,329064917,0.0090,0.0072,0.0071,493597375,-0.0002,0.0087,331636620.0485,0.0078,337482289
4,Fiji,Hindi,English Fijian Fiji Hindi,889953,0.0091,0.0074,0.0072,1334929,-0.0002,0.0089,897300.7759,0.0079,913103
5,Mexico,Spanish,Spanish,127575529,0.0157,0.0141,0.0123,191363293,-0.0003,0.0157,129412041.6788,0.0140,133412001
6,Philippines,Spanish,English Filipino Filipino Sign Language Filipi...,108116615,0.0166,0.0164,0.0151,162174922,-0.0001,0.0168,112276852.9624,0.0160,113748843
7,Egypt,Arabic,Arabic,100388073,0.0182,0.0218,0.0187,150582109,0.0001,0.0193,107176953.3715,0.0196,106731707
8,Iran,Arabic,Persian Persian Persian Arabic,82913906,0.0114,0.0125,0.0104,124370859,-0.0001,0.0119,85254852.4532,0.0114,86017665
9,Malaysia,Malay,Malay,31949777,0.0183,0.0178,0.0135,47924665,-0.0005,0.0189,32253202.9374,0.0165,33664620
